In [9]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [10]:
# Kết nối tới cơ sở dữ liệu Neo4j
from py2neo import Graph
import re

neo4j_graph = Graph("neo4j+s://503c5eaf.databases.neo4j.io", auth=("neo4j", "OplNn6X6ehDa3GhvqJECbHcfXSe_L9juCTK2SGuvDaQ"), name="neo4j")

# Truy vấn lấy thông tin các môn học và các quan hệ ngữ nghĩa bổ sung
query = """
  MATCH (ancestor:Resource {uri: 'http://localhost/ontologies/2024/10/11/edu_program#monHoc'})
  MATCH (n:Resource)-[:rdfs__subClassOf*]->(ancestor)
  MATCH (instance:Resource)-[:rdf__type]->(n)
  OPTIONAL MATCH (instance)-[:ns0__coNoiDung|:ns0__songHanh|:ns0__noiDungCua|:ns0__tienQuyet|:ns0__hocTruoc|:ns0__thuocChuyenNganh]->(relatedInstance)
  RETURN DISTINCT instance.ns0__maMonHoc AS code, instance.rdfs__label AS courseName, elementId(instance) AS elementId,
                  collect(DISTINCT relatedInstance) AS relatedInstances
"""

# Chạy truy vấn và lấy kết quả
results = neo4j_graph.run(query).data()

# Bảng từ điển để chuyển đổi viết tắt thành dạng đầy đủ
abbreviation_dict = {
    "PBL": "Project Based Learning",
    "ATTT": "An toàn thông tin",
    "HTTT": "Hệ thống thông tin",
    "CNPM": "Công nghệ phần mềm"
}

# Hàm để mở rộng các từ viết tắt trong văn bản
def expand_abbreviations(text, abbreviation_dict):
    words = text.split()
    expanded_words = [abbreviation_dict.get(word.upper(), word) for word in words]
    return " ".join(expanded_words)

# Hàm tiền xử lý văn bản để mở rộng các từ viết tắt và chuẩn hóa văn bản
def preprocess_text(text):
    text = text.lower()  # Chuyển thành chữ thường
    text = re.sub(r'[^\w\s]', '', text)  # Loại bỏ dấu câu
    return expand_abbreviations(text, abbreviation_dict)  # Mở rộng từ viết tắt

# Khởi tạo PhoBERT để xử lý ngôn ngữ tự nhiên
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
model = AutoModel.from_pretrained("vinai/phobert-base-v2")

# Hàm Mean Pooling để lấy embedding của câu
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state  # (batch_size, sequence_length, hidden_size)
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Hàm mã hóa câu thành embeddings sử dụng PhoBERT
def encode_sentence(sentence):
    sentence = preprocess_text(sentence)  # Chuyển đổi viết tắt trước khi mã hóa
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, max_length=128, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return mean_pooling(outputs, inputs['attention_mask'])

# Mã hóa các tên môn học thành embeddings
course_embeddings = []
for result in results:
    course_name = result['courseName']
    expanded_course_name = preprocess_text(course_name)  # Chuyển đổi viết tắt thành dạng đầy đủ
    embedding = encode_sentence(expanded_course_name)
    course_embeddings.append((result, embedding))

# Nhập từ khóa tìm kiếm và mã hóa thành embedding
search_query = "PBL6 an toàn thông tin"  # Thay thế bằng từ khóa tìm kiếm
expanded_search_query = preprocess_text(search_query)  # Chuyển đổi viết tắt thành dạng đầy đủ
search_embedding = encode_sentence(expanded_search_query).squeeze(0)

# Tính toán độ tương đồng cosine giữa từ khóa tìm kiếm và các tên môn học
similar_results = []
keyword = "PBL"
for result, embedding in course_embeddings:
    embedding = embedding.squeeze(0)
    similarity = F.cosine_similarity(search_embedding, embedding, dim=0).item()
    # Thêm trọng số cho kết quả chứa từ khóa chính xác
    if keyword.lower() in result['courseName'].lower():
        similarity += 0.1
    result['similarity'] = similarity
    similar_results.append(result)

# Sắp xếp kết quả theo độ tương đồng giảm dần và lấy top 10 kết quả
sorted_filtered_results = sorted(similar_results, key=lambda x: x['similarity'], reverse=True)[:10]

# In ra top 10 kết quả tìm kiếm
for result in sorted_filtered_results:
    print(f"Mã môn học: {result['code']}, Tên môn học: {result['courseName']}, Element ID: {result['elementId']}, Độ tương đồng: {result['similarity']:.4f}")
    # In ra thêm thông tin các quan hệ liên quan
    for related_instance in result['relatedInstances']:
        print(f"  - Liên kết với: {related_instance}")

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mã môn học: mm74, Tên môn học: PBL6: Đồ án chuyên ngành 1 (ATTT), Element ID: 4:e33d50a5-f668-425e-b2d8-a6a2f521b315:104, Độ tương đồng: 0.9943
  - Liên kết với: (_88:Resource:ns0__noiDungAttt:owl__NamedIndividual {rdfs__label: 'N\u1ed9i dung PBL6: \u0110\u1ed3 \u00e1n chuy\u00ean ng\u00e0nh 1 (ATTT)', uri: 'http://localhost/ontologies/2024/10/11/edu_program#ndPbl6DoAnChuyenNganh1ATTT'})
  - Liên kết với: (_102:Resource:ns0__attt:owl__NamedIndividual {ns0__hocKy: 7, ns0__laMonTuChon: false, ns0__maMonHoc: 'mm67', ns0__soTinChi: 2.0, rdfs__label: 'M\u00e3 h\u00f3a v\u00e0 m\u1eadt m\u00e3', uri: 'http://localhost/ontologies/2024/10/11/edu_program#maHoaVaMatMa'})
  - Liên kết với: (_132:Resource:ns0__attt:owl__NamedIndividual {ns0__hocKy: 7, ns0__laMonTuChon: false, ns0__maMonHoc: 'mm66', ns0__soTinChi: 2.0, rdfs__label: 'An to\u00e0n th\u00f4ng tin m\u1ea1ng', uri: 'http://localhost/ontologies/2024/10/11/edu_program#anToanThongTinMang'})
Mã môn học: mm65, Tên môn học: PBL6: Đồ án chuyên